In [16]:
%load_ext autoreload
%autoreload 2

from src.experiment import get_fallacy_df, save_fallacy_df, tuning_train_test_split, get_tuning_examples, save_tuning_examples, TuningSet
from openai import OpenAI

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Fine-tuning

In [17]:
e5_filename = 'data/fallacies_e5.csv'
df_fallacies_e5 = get_fallacy_df(e5_filename, only_incorrect=True)

df_fallacies_e5.head()

[2024-10-28 12:29:09] Created new fallacy identification dataframe.


,step,entity,fallacy,label,category,subcategory
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency


In [20]:
tuning_train_test_split(df_fallacies_e5, n_train=4, n_validation=1)
df_fallacies_e5['tuning'].value_counts()


tuning
test          1160
train          928
validation     232
Name: count, dtype: int64

In [21]:
train_examples = get_tuning_examples(df_fallacies_e5, TuningSet.TRAIN)
validation_examples = get_tuning_examples(df_fallacies_e5, TuningSet.VALIDATION)

save_tuning_examples(train_examples, 'data/tuning-train.jsonl')
save_tuning_examples(validation_examples, 'data/tuning-validation.jsonl')

In [ ]:
client = OpenAI()

client.fine_tuning.jobs.create(
    training_file="data/tuning-train.jsonl",
    validation_file="data/tuning-validation.jsonl",
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    suffix="fallacy-detection-v1"
)